# Import Packages

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchviz import make_dot

from typing import Any, Dict, List, Tuple, Union
from torch.jit.annotations import TensorType

from IPython.display import Markdown as md # For automated updates of the table

# Dataset

In [2]:
# MNIST dataset and dataloaders
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = torchvision.datasets.MNIST(root=os.getcwd(), train=True, transform=transform, download=True)

In [3]:
# Split the dataset into train, val, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [4]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=os.cpu_count())
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=os.cpu_count())
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=os.cpu_count())

# Models
Below is a dictionary of the models used for the experiment, with different numbers of convolutional layers and fully connected layers

In [5]:
# Parameter dictionary for each model and 
# corresponding layer parameters.
params: Dict[str,Dict[str,torch.nn]] = {
    "model.1": {
        "conv_layers": (nn.Conv2d(1, 32, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.Conv2d(32, 64, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.Conv2d(64, 128, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.Conv2d(128, 256, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        ),
        "fc_layer": (nn.Linear(256, 128),
                     nn.ReLU(),
                     nn.Linear(128, 10),)
    },
    "model.2": {
        "conv_layers": (nn.Conv2d(1, 32, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.Conv2d(32, 32, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.Conv2d(32, 32, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.Conv2d(32, 32, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2)
                        ),
        "fc_layer": (nn.Linear(32, 32),
                     nn.ReLU(),
                     nn.Linear(32, 10),)
    },
    "model.3": {
        "conv_layers": (nn.Conv2d(1, 32, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.Conv2d(32, 64, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.Conv2d(64, 128, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2)
                        ),
        "fc_layer": (nn.Linear(128 * 3 * 3, 256),
                     nn.ReLU(),
                     nn.Linear(256, 10))
    },
    "model.4": {
        "conv_layers": (nn.Conv2d(1, 32, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.Conv2d(32, 64, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2)
                        ),
        "fc_layer": (nn.Linear(64 * 7 * 7, 128),
                     nn.ReLU(),
                     nn.Linear(128, 10)
                     )
    },
    "model.5": {
        "conv_layers": (nn.Conv2d(1, 32, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        ),
        "fc_layer": (nn.Linear(32 * 14 * 14, 128),
                     nn.ReLU(),
                     nn.Linear(128, 10)
                     )
    },
    "model.6": {
        "conv_layers": (nn.Conv2d(1, 32, kernel_size=3, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        ),
        "fc_layer": (nn.Linear(14 * 14 * 32, 10))
    },
}

# Define Functions

In [6]:
def count_trainable_parameters(model: pl.LightningModule) -> int:
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params

In [7]:
def _save_and_viz_pl_model(model: Union[nn.Module,pl.LightningModule], filename: str) -> None:
    """Helper function to visualize and plot the model architecture.

    Args:
        model: Input pytorch (lightning) model.
        filename: Output filename (no file extension).
    """
    # Create a dummy input with the same shape as expected input during training
    dummy_input = torch.randn(1, 1, 28, 28)

    # Generate the visualization of the model architecture
    dot = make_dot(model(dummy_input))
    # params = dict(model.named_parameters())

    # Save the visualization as an image
    dot.format = 'png'
    dot.render(filename, cleanup=True)
    
    return None

In [8]:
def save_pt_model(model: Union[nn.Module,pl.LightningModule], filename: str) -> None:
    """Saves pytorch (lightning) model, and creates visualization of model architecture.

    Args:
        model: Input pytorch (lightning) model.
        filename: Output filename.
    """
    # TODO: Save metadata file for the model.
    # Check filename
    filename: str
    ext: str
    
    if ('pt' or 'pth') in filename:
        filename, ext = os.path.splitext(filename)
    else:
        ext: str = ".pt"
    
    # Save model (and model state)
    torch.save(model.state_dict(), f"{filename}{ext}")

    # Save image of model architecture
    _save_and_viz_pl_model(model=model, filename=filename)

    return None

In [9]:
def load_pt_model(model: Union[nn.Module,pl.LightningModule], filename: str) -> Union[nn.Module,pl.LightningModule]:
    """Loads saved/trained model, in which the model class **must** be provided.

    Args:
        model: Input model class objoect.
        filename: Input filename that corresponds to trained saved/trained model.

    Returns:
        Trained model.
    """
    if ('pt' or 'pth') in filename:
        pass
    else:
        filename: str = f"{filename}.pt"

    # model = TheModelClass(*args, **kwargs)
    model.load_state_dict(torch.load(filename))
    model.eval() # sets dropout and batch normalization layers to evaluation mode
    return model

In [10]:
def get_gradient_norms(model: Union[nn.Module,pl.LightningModule], weight: bool = True, bias: bool = False) -> List[Tuple[str,float]]:
    model.eval()
    sample_input = torch.randn(1, 1, 28, 28)  # Replace with your own sample input
    outputs = model(sample_input)
    loss = torch.sum(outputs)  # Create a dummy loss

    # Backpropagate to compute gradients
    loss.backward()

    # Compute gradient norms
    gradient_norms: List[Tuple[str,float]] = []
    for name, param in model.named_parameters():
        if param.grad is not None:
            if weight and ('weight' in name):
                gradient_norms.append((name, param.grad.norm().item()))
            
            if bias and ('bias' in name):
                gradient_norms.append((name, param.grad.norm().item()))
                
    return gradient_norms

In [11]:
def get_layerwise_norms(model: Union[nn.Module,pl.LightningModule], weight: bool = True, bias: bool = False) -> List[Tuple[str,float]]:
    model.eval()
    sample_input = torch.randn(1, 1, 28, 28)  # Replace with your own sample input
    outputs = model(sample_input)
    loss = torch.sum(outputs)  # Create a dummy loss

    # Backpropagate to compute gradients
    loss.backward()
    
    layerwise_norms: List[Tuple[str,float]] = []
    for name, param in model.named_parameters():
        layer_name = name #.split('.')[0]  # Extract the layer name
        norm = param.norm().item()

        if weight and ('weight' in name):
            layerwise_norms.append((layer_name, norm))
        
        if bias and ('bias' in name):
            layerwise_norms.append((layer_name, norm))
            
    return layerwise_norms

In [12]:
def calculate_total_parameter_norm(model):
    total_norm = 0.0
    for param in model.parameters():
        total_norm += param.norm().item()
    return total_norm

In [13]:
def calculate_parameter_norms_per_layer(model, weight: bool = True, bias: bool = False):
    norms_per_layer = {}
    for name, param in model.named_parameters():
        layer_name = name #.split('.')[0]  # Extract the layer name
        norm = param.norm().item()

        if weight and ('weight' in name):
            if layer_name not in norms_per_layer:
                norms_per_layer[layer_name] = []
            norms_per_layer[layer_name].append(norm)
        
        if bias and ('bias' in name):
            if layer_name not in norms_per_layer:
                norms_per_layer[layer_name] = []
            norms_per_layer[layer_name].append(norm)
    return norms_per_layer

# Define Neural Network

In [14]:
# Define the LightningModule
class ConvNet(pl.LightningModule):
    def __init__(self, params: Dict[str,torch.nn]):
        super(ConvNet, self).__init__()
        self.save_hyperparameters()
        self.conv_layers = nn.Sequential(
            *params.get('conv_layers')
        )
        
        try:
            self.fc_layer = nn.Sequential(
                *params.get('fc_layer')
            )
        except TypeError:
            self.fc_layer = nn.Sequential(
                params.get('fc_layer')
            )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)
    
    # Define the training step method
    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = nn.CrossEntropyLoss()(outputs, targets)
        self.log('train_loss', loss)
        return loss
    
    # Define the validation step method
    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = nn.CrossEntropyLoss()(outputs, targets)
        self.log('val_loss', loss, prog_bar=True)  # Logging the validation loss
    
    # Define the test step method
    def test_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = nn.CrossEntropyLoss()(outputs, targets)
        self.log('test_loss', loss)  # Logging the test loss
        preds = torch.argmax(outputs, dim=1)
        acc = (preds == targets).float().mean()
        self.log('test_acc', acc, prog_bar=True)  # Logging the test accuracy

# Train Models

In [15]:
# Create trainer object
trainer = pl.Trainer(accelerator='mps',max_epochs=10,devices=1)  # Set max_epochs and gpus according to your environment

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
MARKDOWN: str = ""

for param in params.keys():
    # Print Model number to screen
    print(f"\n{param}:\n")
    
    # Initialize the Lightning Trainer
    model = ConvNet(params=params.get(param))

    # Train the model using PyTorch Lightning
    trainer.fit(model, train_loader, val_loader)

    # Save trained model
    save_pt_model(model=model,filename=f"models/{param}")

    MARKDOWN += f"### {param}: \n\n![](models/{param}.png)\n\n"

Missing logger folder: /Users/adebayobraimah/Desktop/projects/fall2023project/lightning_logs

  | Name        | Type       | Params
-------------------------------------------
0 | conv_layers | Sequential | 387 K 
1 | fc_layer    | Sequential | 34.2 K
-------------------------------------------
422 K     Trainable params
0         Non-trainable params
422 K     Total params
1.688     Total estimated model params size (MB)



model.1:



Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
/Users/adebayobraimah/Desktop/projects/fall2023project/.env/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /Users/adebayobraimah/Desktop/projects/fall2023project/lightning_logs/version_0/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name        | Type       | Params
-------------------------------------------
0 | conv_layers | Sequential | 320   
1 | fc_layer    | Sequential | 62.7 K
-------------------------------------------
63.1 K    Trainable params
0         Non-trainable params
63.1 K    Total params
0.252     Total estimated model params size (MB)



model.6:



Sanity Checking: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [17]:
# Show model architecture diagrams
md(MARKDOWN)

### model.1: 

![](models/model.1.png)

### model.2: 

![](models/model.2.png)

### model.3: 

![](models/model.3.png)

### model.4: 

![](models/model.4.png)

### model.5: 

![](models/model.5.png)

### model.6: 

![](models/model.6.png)



# Obtain Models' Metrics

In [18]:
results_dict: Dict[str,Dict[str,Any]] = {}

In [19]:
# Norm calcuation booleans
weight: bool = True
bias: bool = False

for param in params.keys():
    # Print Model number to screen
    print(f"\n{param}:\n")

    tmp_dict = {}

    # Load model
    model = load_pt_model(model=ConvNet(params=params.get(param)),filename=f"models/{param}")
    # trainer = pl.Trainer(accelerator='mps',max_epochs=10,devices=1)  # Set max_epochs and gpus according to your environment

    # Evaluate the model on the test data
    print(f"Training accuracy:")
    train_results = trainer.test(model, dataloaders=train_loader)

    # Evaluate the model on the test data
    print(f"Testing accuracy:")
    test_results = trainer.test(model, dataloaders=test_loader)

    # Countable parameters
    print(f"Number of trainable parameters (weights): {count_trainable_parameters(model=model):,}")
    countable_parameters: str = f"{count_trainable_parameters(model=model):,}"

    # Gradient Norms
    grad_norms = get_gradient_norms(model=model, weight=weight, bias=bias)

    # Layerwise Norms
    layer_norms = get_layerwise_norms(model=model, weight=weight, bias=bias)

    # Total parameter norm
    total_norm = calculate_total_parameter_norm(model=model)

    # Parameter norms per layer
    norms_per_layer = calculate_parameter_norms_per_layer(model=model, weight=weight, bias=bias)

    tmp_dict = {
        "train_acc": f"{train_results[0].get('test_acc'):.4f}",
        "test_acc": f"{test_results[0].get('test_acc'):.4f}",
        "parameters": countable_parameters,
        "grad_norm": grad_norms,
        "layer_norm": layer_norms,
        "total_norm": f"{total_norm:.4f}",
        "norms_per_layer": norms_per_layer,
    }

    results_dict.update({param:tmp_dict})


model.1:

Training accuracy:


/Users/adebayobraimah/Desktop/projects/fall2023project/.env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:480: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9976666569709778     │
│         test_loss         │   0.006960821803659201    │
└───────────────────────────┴───────────────────────────┘

Testing accuracy:


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.11531250178813934    │
│         test_loss         │    2.3989295959472656     │
└───────────────────────────┴───────────────────────────┘

Testing accuracy:


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.11683333665132523    │
│         test_loss         │      2.3984534740448      │
└───────────────────────────┴───────────────────────────┘

Number of trainable parameters (weights): 63,050


In [20]:
MARKDOWN = """

Table of metrics for each of the models defined above.

| model \ specifications | Train Accuracy | Test Accuracy | Number of Trainable Parameters | Gradient Norm | Layerwise Norm | Total Parameter Norm | Per Layer Parameter Norm |
|------------------------|----------------|---------------|--------------------------------|---------------|----------------|----------------------|--------------------------|
"""

In [21]:
# Fill in markdown table
for name, metric in results_dict.items():
    # print(f"{name}: {metric}")
    MARKDOWN += f"| **{name}** | {metric.get('train_acc')} | {metric.get('test_acc')} | {metric.get('parameters')} | {metric.get('grad_norm')} | {metric.get('layer_norm')} | {metric.get('total_norm')} | {metric.get('norms_per_layer')} |\n"

In [22]:
# Show table of metrics for each model
md(MARKDOWN)



Table of metrics for each of the models defined above.

| model \ specifications | Train Accuracy | Test Accuracy | Number of Trainable Parameters | Gradient Norm | Layerwise Norm | Total Parameter Norm | Per Layer Parameter Norm |
|------------------------|----------------|---------------|--------------------------------|---------------|----------------|----------------------|--------------------------|
| **model.1** | 0.9977 | 0.9917 | 422,026 | [('conv_layers.0.weight', 8.96180534362793), ('conv_layers.3.weight', 17.618816375732422), ('conv_layers.6.weight', 27.507503509521484), ('conv_layers.9.weight', 30.44315528869629), ('fc_layer.0.weight', 21.991668701171875), ('fc_layer.2.weight', 39.368282318115234)] | [('conv_layers.0.weight', 4.07852840423584), ('conv_layers.3.weight', 12.439067840576172), ('conv_layers.6.weight', 19.17218017578125), ('conv_layers.9.weight', 28.12639617919922), ('fc_layer.0.weight', 10.133509635925293), ('fc_layer.2.weight', 2.6812291145324707)] | 79.8994 | {'conv_layers.0.weight': [4.07852840423584], 'conv_layers.3.weight': [12.439067840576172], 'conv_layers.6.weight': [19.17218017578125], 'conv_layers.9.weight': [28.12639617919922], 'fc_layer.0.weight': [10.133509635925293], 'fc_layer.2.weight': [2.6812291145324707]} |
| **model.2** | 0.1040 | 0.1063 | 29,450 | [('conv_layers.0.weight', 0.2227998971939087), ('conv_layers.3.weight', 1.7035162448883057), ('conv_layers.6.weight', 2.1910617351531982), ('conv_layers.9.weight', 1.9787214994430542), ('fc_layer.0.weight', 0.6428667902946472), ('fc_layer.2.weight', 1.481598138809204)] | [('conv_layers.0.weight', 3.321035385131836), ('conv_layers.3.weight', 3.2883975505828857), ('conv_layers.6.weight', 3.2639966011047363), ('conv_layers.9.weight', 3.258723258972168), ('fc_layer.0.weight', 3.2014241218566895), ('fc_layer.2.weight', 1.799263596534729)] | 20.7945 | {'conv_layers.0.weight': [3.321035385131836], 'conv_layers.3.weight': [3.2883975505828857], 'conv_layers.6.weight': [3.2639966011047363], 'conv_layers.9.weight': [3.258723258972168], 'fc_layer.0.weight': [3.2014241218566895], 'fc_layer.2.weight': [1.799263596534729]} |
| **model.3** | 0.0647 | 0.0672 | 390,410 | [('conv_layers.0.weight', 0.505016565322876), ('conv_layers.3.weight', 3.9824531078338623), ('conv_layers.6.weight', 6.05840539932251), ('fc_layer.0.weight', 11.304313659667969), ('fc_layer.2.weight', 5.046265602111816)] | [('conv_layers.0.weight', 3.2629506587982178), ('conv_layers.3.weight', 4.631279468536377), ('conv_layers.6.weight', 6.531067848205566), ('fc_layer.0.weight', 9.2524995803833), ('fc_layer.2.weight', 1.8152220249176025)] | 27.5682 | {'conv_layers.0.weight': [3.2629506587982178], 'conv_layers.3.weight': [4.631279468536377], 'conv_layers.6.weight': [6.531067848205566], 'fc_layer.0.weight': [9.2524995803833], 'fc_layer.2.weight': [1.8152220249176025]} |
| **model.4** | 0.0901 | 0.0898 | 421,642 | [('conv_layers.0.weight', 0.9435096979141235), ('conv_layers.3.weight', 6.933793544769287), ('fc_layer.0.weight', 25.223155975341797), ('fc_layer.2.weight', 5.808513641357422)] | [('conv_layers.0.weight', 3.174894094467163), ('conv_layers.3.weight', 4.625846862792969), ('fc_layer.0.weight', 6.535171031951904), ('fc_layer.2.weight', 1.7795995473861694)] | 17.7278 | {'conv_layers.0.weight': [3.174894094467163], 'conv_layers.3.weight': [4.625846862792969], 'fc_layer.0.weight': [6.535171031951904], 'fc_layer.2.weight': [1.7795995473861694]} |
| **model.5** | 0.0816 | 0.0823 | 804,554 | [('conv_layers.0.weight', 1.9999253749847412), ('fc_layer.0.weight', 69.93987274169922), ('fc_layer.2.weight', 9.95755672454834)] | [('conv_layers.0.weight', 3.1836984157562256), ('fc_layer.0.weight', 6.530889511108398), ('fc_layer.2.weight', 1.815815806388855)] | 12.8820 | {'conv_layers.0.weight': [3.1836984157562256], 'fc_layer.0.weight': [6.530889511108398], 'fc_layer.2.weight': [1.815815806388855]} |
| **model.6** | 0.1153 | 0.1168 | 63,050 | [('conv_layers.0.weight', 5.318680763244629), ('fc_layer.0.weight', 179.98753356933594)] | [('conv_layers.0.weight', 3.260796070098877), ('fc_layer.0.weight', 1.8255901336669922)] | 6.0232 | {'conv_layers.0.weight': [3.260796070098877], 'fc_layer.0.weight': [1.8255901336669922]} |
